## Q3b. image (20 marks)

### Instructions

In part b, we want to create a "painting" like what we have seen in the instructions of part a:

<img src="../figs/purple_soup.png" width=300/>

Note:
* Part b has 3 parts. You may choose not to submit part 1 as it is an intermediate step to solve part 2
* You may find your code for Q3a part 3, Q3b part 1 and Q3b part 2 is quite similar. No need to worry about "copying and pasting code" for these parts
    * The similarity is due to the fact that Q3a part 3 and Q3b part 1 are "simplified" version of the original problem to be solved by Q3b part 2

---

1. (12 marks) Write the function definition for the function `apply_filter()` to replace the colour of a given image with the given colours. 

    The function takes an image (non-empty 3d `np.ndarray` with dtype `np.uint8`), and a `tuple` of 1d `np.ndarray` with dtype `np.uint8` with length 3 representing the RGB values of colours.

    The function returns a _new_ image (3d `np.ndarray` with dtype `np.uint8`) that is similar to the original image, but with the colours of the image replaced by the given tuple of colours. For each pixel, its colour is replaced by the closest out of the given colours in terms of Euclidean distance.

    * Example: Suppose the given tuple representing colours `(0, 1, 2), (0, 180, 255), (255, 254, 253), (0, 0, 0)` and the pixel of interest has RGB(214, 234, 248). 

        The distance between the pixel and the first colour in the given tuple is:
        $$ dist((214, 234, 248),(0, 1, 2)) = \sqrt{(214 - 0)^2 + (234 - 1)^2 + (248 - 2)^2} = 400.75$$

        The distance between the pixel and the second colour in the given tuple is:
        $$ dist((214, 234, 248),(0, 180, 255)) = \sqrt{(214 - 0)^2 + (234 - 180)^2 + (248 - 255)^2} = 220.8$$

        The distance between the pixel and the third colour in the given tuple is:
        $$ dist((214, 234, 248),(255, 254, 253)) = \sqrt{(214 - 255)^2 + (234 - 254)^2 + (248 - 253)^2} = 45.89$$

        The distance between the pixel and the fourth colour in the given tuple is:
        $$ dist((214, 234, 248),(0, 0, 0)) = \sqrt{(214 - 0)^2 + (234 - 0)^2 + (248 - 0)^2} = 402.56$$

        As the pixel is closest to the third colour in the given tuple, it will be replaced by the third colour (255, 254, 253).

    The shape of the returned `np.ndarray` is the same as the given images. If there is a tie in terms of the distance, any of the closest colours can be used.

    * Hint: 
        * You may want to first calculate the distance between the given image with each colour, then for each colour find out which pixels to replace
        * You may want to use [`np.min()`](https://numpy.org/doc/stable/reference/generated/numpy.min.html) or [`np.argmin()`](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html) providing the optional argument `axis=...`
            * To understand how `axis=...` works in general, see the help slides for workshop 7, "2d array: Element-wise statistics" and Exercise 7.7 from Lecture 7 and the "Element-wise statistics" from Lecture 8 
            * Feel free to learn about how to use `min()` and `argmin()` by searching online or having a look of the official document
    * Note: 
        * You can have a small number of iterations (e.g. iterate over each colour), but no large number of iterations (e.g. iterate over each pixel of the given image) is needed
        * If you find the problem difficult to solve, restricting to having at most 3 colours in the given tuple instead 
        * The image given as an argument should not be changed
    * Example usage:

    <img src="../figs/demo_filter_1.png" width=1000/>

In [ ]:
## please write the function definition for apply_filter() here. You can also skip this and attempt part 2 directly
## for this part, no need to worry about repeated code part 2, or Q3a part 3

---

2. (6 marks) Rewrite the function definition for the function `apply_filter()` to replace the colour of a given image with the given colours. 

    The function takes an image (non-empty 3d `np.ndarray` with dtype `np.uint8`), a `tuple` of 1d `np.ndarray` with dtype `np.uint8` with length 3 representing the RGB values of colours, and another tuple of 1d `np.ndarray` with dtype `np.uint8` with length 3 representing the RGB values of colours. The last argument is _optional_. The two tuples must be at the same length if the last tuple is provided.

    If the optional argument is not provided, it should work the same as it is in part 1. If the optional argument is provided, for each pixel, the colour to replace is the colour in the second tuple that corresponds to the colour that is closest out of the colours in the first tuple in terms of Euclidean distance.
    
* Example: Suppose the first given tuple representing colours `(0, 1, 2), (0, 180, 255), (255, 254, 253), (0, 0, 0)` and the pixel of interest has RGB(214, 234, 248) (i.e. same as the example in part 1), and the second given tuple representing colours `(7, 8, 9), (0, 180, 255), (10, 10, 10), (0, 0, 0)`

    As the pixel is closest to the third colour in the first given tuple, it will be replaced by the third colour in the second given tuple (i.e. (10, 10, 10)).

    * Note: 
        * You can have a small number of iterations (e.g. iterate over each colour), but no large number of iterations (e.g. iterate over each pixel of the given image) is needed
        * The image given as an argument should not be changed 
    * Example usage:

    <img src="../figs/demo_filter_2.png" width=1000/>
    
    * Your implementation:

In [ ]:
## please write the function definition for apply_filter() here
## for this part, no need to worry about repeated code with part 1, or Q3a part 3

---

3. (2 marks) Please load in some images and apply `apply_filter()` and `repetition()` from `image_util` to the images to make something like what we have at the beginning of the instruction. Please display the resulting images.

In [ ]:
## your code

---

## Requirements

* You cannot use any modules and libraries (except `NumPy`) to answer this question
* This question is designed to assess your ability to manipulate `NumPy` array with vectorised operations and indexing/slicing - therefore, please use `NumPy` and vectorise operations _wherever appropriate and possible_
* You can only use functionalities from `NumPy` that have been demonstrated in the lecture, except it is stated otherwise in the question